## 데이터 수집

In [189]:
import pandas as pd
df_titanic = pd.read_csv('../../datasets/titanic_disaster_train.csv')
df_titanic.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


## 데이터 분석

## 데이터 전처리

### 결측치 처리

In [190]:
df_titanic.dropna(inplace = True)
# 결측치 제거: 데이터를 사용할 때 결측값이 있는 행(row)을 제거합니다.

### 스케일링
- 지수화와 비슷

## 데이터 분할

In [191]:
features = df_titanic.drop(columns=['Survived', 'PassengerId', 'Name'
                                    , 'Sex', 'Ticket', 'Cabin', 'Embarked'])
label = df_titanic['Survived']
features.columns

# object 는 공식이 될 수 없기 때문에 빼야한다

Index(['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

## 모델 학습

### 비선형 모델
- SVM(Support Vector Machine) : 고차원 공간에서 데이터를 비선형 분류해 표현
 
    + 핵심개념 : 데이터를 단순히 선으로 나눌 수 없을 때(비선형 데이터) 사용하는 모델
    
    + 예를 들어, 데이터가 둥근 모양으로 분포하면 직선으로 나눌 수 없죠. 이때 SVM(Support Vector Machine)은 데이터를 더 높은 차원으로 옮겨, 직선으로 나눌 수 있게 도와줍니다.
    
    + 이것을 "고차원 공간"이라고 부르며 이렇게 하면 비선형적으로 분포된 데이터도 잘 분류할 수 있습니다.

In [192]:
from sklearn.svm import SVC
model_svc = SVC(probability = True
                , decision_function_shape = 'ovr')
model_svc.fit(X = features, y = label)
'''
SVC(Support Vector Classifier): SVM을 사용하여 데이터를 분류하는 모델입니다.

파라미터 설명:
   # probability=True: 확률값을 계산할 수 있도록 설정합니다.
   # decision_function_shape='ovr': "One-vs-Rest" 방식을 사용해 다중 클래스 분류를 처리합니다.

fit: 주어진 데이터(features)와 라벨(label)을 이용해 모델을 학습시킵니다
'''

'\nSVC(Support Vector Classifier): SVM을 사용하여 데이터를 분류하는 모델입니다.\n\n파라미터 설명:\n   # probability=True: 확률값을 계산할 수 있도록 설정합니다.\n   # decision_function_shape=\'ovr\': "One-vs-Rest" 방식을 사용해 다중 클래스 분류를 처리합니다.\n\nfit: 주어진 데이터(features)와 라벨(label)을 이용해 모델을 학습시킵니다\n'

## 모델 평가

### ROC, AUC 평가 방식 사용
- predict probablly 값 중에 양성 클래스 확률 값을 사용

- ROC (Receiver Operating Characteristic Curve):

+ 모델의 성능을 시각적으로 평가하기 위한 곡선입니다.
    + x축: FPR (False Positive Rate)
        + → 잘못 양성으로 예측한 비율.
    + y축: TPR (True Positive Rate)
        + → 실제 양성을 얼마나 잘 맞췄는지.

+ AUC (Area Under Curve):
    + ROC 곡선 아래의 면적입니다.
        + 1에 가까울수록 모델이 성능이 좋습니다.

#### label 카테고리가 2개 일때

In [193]:
from sklearn.metrics import roc_curve, auc 

In [194]:
predict_score = model_svc.predict_proba(features)[ : , 1]
predict_score[:3]

# predict_proba: 예측값 대신 클래스별 확률값을 반환합니다.
# → [:, 1]: 양성(positive) 클래스의 확률값만 가져옵니다.

array([0.70034646, 0.69765465, 0.65449378])

In [195]:
fpr, tpr, threadhold = roc_curve(y_true = label, y_score = predict_score)
# fpr, tpr, threadhold
fpr.shape, tpr.shape, threadhold.shape

'''
# roc_curve:
    → 모델이 특정 임계값에서 FPR과 TPR을 계산해 반환합니다.
    fpr: 잘못된 예측(거짓 양성 비율).
    tpr: 정확한 예측(참 양성 비율).
    threshold: 예측 확률의 기준값.
'''

'\n# roc_curve:\n    → 모델이 특정 임계값에서 FPR과 TPR을 계산해 반환합니다.\n    fpr: 잘못된 예측(거짓 양성 비율).\n    tpr: 정확한 예측(참 양성 비율).\n    threshold: 예측 확률의 기준값.\n'

In [196]:
# AUC(Area Under Curve) 구하기 : 면적 값이 1에 가까우면 좋음
auc_score = auc(fpr, tpr)
auc_score

0.7062330623306233

#### label 카테고리 갯수가 3개 이상인 경우
- OVR(One vs Rest) : 번갈아 하나에 양성 클래스와 음성 클래스 조합, 최종 높은 것 선택

- 다중 클래스를 처리하기 위해 OVR 방식을 사용합니다.
    + OVR: 여러 클래스 중 하나를 양성 클래스로 간주하고 나머지는 음성으로 처리.
    + 모든 클래스에 대해 이를 반복한 후, 가장 높은 확률을 최종 예측값으로 선택.

In [197]:
from sklearn.datasets import load_iris
iris_dataset = load_iris()
label = iris_dataset['target']
features = iris_dataset['data']
label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [198]:
model_svc.fit(X = features, y = label)

SVC(probability=True)

In [199]:
predict_score = model_svc.predict_proba(features)
predict_score[:3]

array([[0.97765257, 0.01439786, 0.00794957],
       [0.96947809, 0.02156327, 0.00895864],
       [0.97876467, 0.01350286, 0.00773246]])

In [200]:

from sklearn.metrics import roc_auc_score
auc_score = roc_auc_score(y_true = label
                                , y_score = predict_score
                                , multi_class = 'ovr')
auc_score

# roc_auc_score: 다중 클래스의 AUC 점수를 계산합니다.
# multi_class='ovr': OVR 방식을 사용하여 각 클래스별로 AUC를 계산합니다.


0.9981999999999999

## 모델 배포